# Ocean Heat Contant

$OHC = \Delta T \times c_p \times \rho \times V$

unit conversion to SI
- $c_p$: erg/g/K = 1e-7J / 1e-3kg / K = 1e-4 J/kg/K $\rightarrow$ 3996 J/kg/K
- $\rho$: g/cm^3 = 1e3 kg/m^3

In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import OHC_integrals
from maps import map_ocn_robinson
from paths import CESM_filename, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rcp, path_results
from regions import boolean_mask, regions_dict
from constants import cp_sw
from timeseries import IterateOutputCESM
from xr_integrate import  xr_vol_int
from xr_regression import xr_linear_trend, xr_linear_trends_2D
from xr_DataArrays import create_xr_DataArray, xr_DZ, xr_AREA, xr_HTN, xr_LATS

In [ ]:
ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
MASK = ds.REGION_MASK

In [ ]:
km = 42

In [ ]:
ctrl = xr.open_dataset(f'{path_samoc}/OHC_integrals_ctrl_Global_Ocean.nc', decode_times=False)
rcp  = xr.open_dataset(f'{path_samoc}/OHC_integrals_rcp_Global_Ocean.nc' , decode_times=False)

In [ ]:
ctrl_t = np.array(ctrl.time.values/365, dtype=int)
rcp_t  = np.array(rcp.time.values/365 , dtype=int)
n_ctrl, n_rcp = len(ctrl_t), len(rcp_t)

In [ ]:
DZT = xr_DZ('ocn')

In [ ]:
dz_mean = DZT.where(DZT>0).mean(dim=('nlat', 'nlon'))

In [ ]:
dz_mean.plot()

In [ ]:
tdepth = ctrl.z_t/100

### remaining issues
- ctrl year 205: too large global value

In [ ]:
ctrl.OHC_global[5]        = (ctrl.OHC_global[4]+ctrl.OHC_global[6])/2
ctrl.OHC_global_levels[5] = (ctrl.OHC_global_levels[4]+ctrl.OHC_global_levels[6])/2
ctrl.OHC_zonal[5]         = (ctrl.OHC_zonal[4]+ctrl.OHC_zonal[6])/2
ctrl.OHC_zonal_levels[5]  = (ctrl.OHC_zonal_levels[4]+ctrl.OHC_zonal_levels[6])/2

In [ ]:
OHC_global_init = ctrl.OHC_global[0].item()

In [ ]:
ctrl.OHC_global.time.values

In [ ]:
ctrl_times = np.arange(0,len(ctrl_t),1)
rcp_times  = np.arange(0,len(rcp_t),1)
lin_fit  = np.polyfit(ctrl_times, ctrl.OHC_global, 1)
quad_fit = np.polyfit(rcp_times , rcp.OHC_global , 2)
ctrl_global_lin_detr = ctrl.OHC_global-(lin_fit[1] + lin_fit[0]*ctrl_times)
rcp_global_lin_detr  = rcp.OHC_global -(lin_fit[1] + lin_fit[0]*rcp_times)
rcp_global_quad_detr = rcp.OHC_global -(quad_fit[2]+quad_fit[1]*rcp_times+quad_fit[0]*rcp_times**2)

## global integral

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot((ctrl.OHC_global.values-OHC_global_init)/1e21, lw=3, c=f'C{0}', label='CTRL')
plt.plot((rcp.OHC_global.values -OHC_global_init)/1e21, lw=3, c=f'C{1}', label='RCP' )
plt.plot(ctrl_global_lin_detr/1e21,        ls='--', lw=3, c=f'C{0}', label='CTRL lin. detrended')
plt.plot(rcp_global_lin_detr /1e21,        ls='--', lw=3, c=f'C{1}', label='RCP detrended \nw/ lin. CTRL trend')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('Ocean Heat Content [ZJ]', fontsize=16)
plt.legend(fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_global_timeseries')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(ctrl_global_lin_detr/1e21, lw=3, label='CTRL lin. detrended')
plt.plot(rcp_global_quad_detr/1e21, lw=3, label='RCP quad. detrended')
plt.plot(ctrl_times[1:], (ctrl.OHC_global.values[1:]-ctrl.OHC_global.values[:-1])/1e21, ls=':', lw=2, c=f'C{0}', label='$\Delta$OHC CTRL')
plt.plot(rcp_times[1:] , (rcp.OHC_global.values[1:] -rcp.OHC_global.values[:-1] )/1e21, ls=':', lw=2, c=f'C{1}', label='$\Delta$OHC RCP')
plt.ylim((-15,25))
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('detrended OHC, OHC change [ZJ]', fontsize=16)
plt.legend(loc=2, fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_global_timeseries_detrended')

## Levels

In [ ]:
# trends in J/m/year
ctrl_global_levels_trend = xr_linear_trend(ctrl.OHC_global_levels/dz_mean)*365
rcp_global_levels_trend  = xr_linear_trend(rcp.OHC_global_levels /dz_mean)*365

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axvline(0, c='k', lw=.5)
plt.plot(ctrl_global_levels_trend, -tdepth/1e3, lw=3, label='CTRL')
plt.plot(rcp_global_levels_trend , -tdepth/1e3, lw=3, label='RCP')
plt.xlabel('OHC trend [J/m/yr]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.legend(fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_trend_depth')

In [ ]:
ctrl_global_level_lin_fit = np.zeros((2, km))
rcp_global_level_lin_fit  = np.zeros((2, km))
rcp_global_level_quad_fit = np.zeros((3, km))
for k in range(km):
    ctrl_global_level_lin_fit[:,k] = np.polyfit(ctrl_times, ctrl.OHC_global_levels[:,k]/dz_mean[k], 1)
    rcp_global_level_lin_fit[:,k]  = np.polyfit(rcp_times , rcp.OHC_global_levels[:,k] /dz_mean[k], 1)
    rcp_global_level_quad_fit[:,k] = np.polyfit(rcp_times , rcp.OHC_global_levels[:,k] /dz_mean[k], 2)
f,ax = plt.subplots(1,2)
ax[0].plot(ctrl_global_level_lin_fit[0,:])
ax[0].plot(rcp_global_level_lin_fit[0,:])
ax[0].set_xlabel('slope')
ax[1].plot(ctrl_global_level_lin_fit[1,:])
ax[1].plot(rcp_global_level_lin_fit[1,:])
ax[1].set_xlabel('offset')

In [ ]:
ctrl_global_levels_lin = np.zeros((n_ctrl, km))
rcp_global_levels_lin  = np.zeros((n_rcp, km))
rcp_global_levels_quad = np.zeros((n_rcp, km))
for t in range(n_ctrl):
    ctrl_global_levels_lin[t,:] = ctrl_global_level_lin_fit[0,:]*t + ctrl_global_level_lin_fit[1,:]
    if t<n_rcp:
        rcp_global_levels_lin[t,:] = rcp_global_level_lin_fit[0,:]*t + rcp_global_level_lin_fit[1,:]
        rcp_global_levels_quad[t,:] = rcp_global_level_quad_fit[0,:]*t**2 + rcp_global_level_quad_fit[1,:]*t + rcp_global_level_quad_fit[2,:]
        

In [ ]:
X_ctrl       , Y_ctrl         = np.meshgrid(ctrl_times, -tdepth/1e3)
X_ctrl_detail, Y_ctrl_detail  = np.meshgrid(ctrl_times, -tdepth[:20]/1e3)
# ctrl_change_vs_mean_ctrl = ((ctrl.OHC_global_levels[:,:]- ctrl.OHC_global_levels.mean(dim='time'))/dz_mean)
ctrl_change_detr = ((ctrl.OHC_global_levels[:,:]/dz_mean- ctrl_global_levels_lin))

f,ax = plt.subplots(2,1, figsize=(8,5), sharex=True)
ax[0].pcolormesh(X_ctrl_detail, Y_ctrl_detail, ctrl_change_detr[:,:20].T/1e21, cmap='RdBu_r', vmin=-.2, vmax=.2)
ax[1].pcolormesh(X_ctrl       , Y_ctrl       , ctrl_change_detr.T/1e21       , cmap='RdBu_r', vmin=-.2, vmax=.2)
ax[1].set_xlabel('time [years]', fontsize=16)
for i in range(2):
    ax[i].set_ylabel('depth [km]', fontsize=16)
plt.tight_layout()
f.align_labels()

In [ ]:
X_rcp       , Y_rcp         = np.meshgrid(rcp_times, -tdepth/1e3)
X_rcp_detail, Y_rcp_detail  = np.meshgrid(rcp_times, -tdepth[:20]/1e3)
# rcp_change_vs_mean_ctrl = ((rcp.OHC_global_levels[:,:]- ctrl.OHC_global_levels.mean(dim='time'))/dz_mean)
rcp_change_vs_mean_ctrl = ((rcp.OHC_global_levels[:,:]/dz_mean - rcp_global_levels_lin))
# rcp_change_detr = ((rcp.OHC_global_levels[:,:]/dz_mean - rcp_global_levels_quad))


f,ax = plt.subplots(2,1, figsize=(8,5), sharex=True)
ax[0].pcolormesh(X_rcp_detail, Y_rcp_detail, rcp_change_detr[:,:20].T/1e21, cmap='RdBu_r', vmin=-.2, vmax=.2)
ax[1].pcolormesh(X_rcp       , Y_rcp       , rcp_change_detr.T/1e21       , cmap='RdBu_r', vmin=-.2, vmax=.2)
ax[1].set_xlabel('time [years]', fontsize=16)
for i in range(2):
    ax[i].set_ylabel('depth [km]', fontsize=16)
plt.tight_layout()
f.align_labels()

# Spatial trends

In [ ]:
%%time
ctrl_vert_trend = xr_linear_trends_2D(da=ctrl.OHC_vertical, dim_names=('nlat', 'nlon'))

In [ ]:
ctrl_vert_diff = (ctrl.OHC_vertical[40:50,:,:].mean(dim='time') - ctrl.OHC_vertical[:10,:,:].mean(dim='time')).where(MASK>0)
rcp_vert_diff  = (rcp.OHC_vertical[40:50,:,:].mean(dim='time')  - rcp.OHC_vertical[:10,:,:].mean(dim='time') ).where(MASK>0)

In [ ]:
label='$\Delta$OHC [J/m$^2$]'; maxv = 7e9

In [ ]:
ctrl_vert_diff

In [ ]:
fn = f'{path_results}/OHC/OHC_trend_vert_int_map_ctrl'
f = map_ocn_robinson(ctrl_vert_diff, 'RdBu_r', minv=-maxv, maxv=maxv, label=label, filename=fn, grid='U')

In [ ]:
fn = f'{path_results}/OHC/OHC_trend_vert_int_map_rcp'
f = map_ocn_robinson(rcp_vert_diff , 'RdBu_r', minv=-maxv, maxv=maxv, label=label, filename=fn, grid='U')

### hiatus spatial pattern
change between year 20 and 28

In [ ]:
rcp_vert_hiatus = (ctrl.OHC_vertical[28,:,:] - ctrl.OHC_vertical[18,:,:]).where(MASK>0)

In [ ]:
f = map_ocn_robinson(rcp_vert_hiatus , 'RdBu_r', minv=-maxv, maxv=maxv, label=label, filename=None, grid='U')

## latitude

In [ ]:
ctrl_zonal_fit = np.polyfit(ctrl_times, ctrl.OHC_zonal[:,11:], 1)  # avoiding NaNs in Southernmost lat bins
rcp_zonal_fit  = np.polyfit(rcp_times , rcp.OHC_zonal[:,11:] , 1)

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(ctrl.TLAT_bins, ctrl.OHC_zonal.mean(dim='time').values, lw=3, label='CTRL mean')
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('zonal OHC integral [J/m]', fontsize=16)
plt.legend(fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_mean')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(ctrl.TLAT_bins[11:], ctrl_zonal_fit[0,:]*365, lw=3, label='CTRL')
plt.plot(ctrl.TLAT_bins[11:], rcp_zonal_fit[0,:] *365, lw=3, label='RCP')
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('zonal OHC linear trend [J/m/yr]', fontsize=16)
plt.legend(fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_trend')

## latitude - depth

In [ ]:
ctrl_zonal_levels_trend = xr_linear_trends_2D(ctrl.OHC_zonal_levels[:,:,11:], dim_names=('TLAT_bins', 'z_t'))*365  # [day^-1] to [yr^-1]
rcp_zonal_levels_trend  = xr_linear_trends_2D(rcp.OHC_zonal_levels[:,:,11:],  dim_names=('TLAT_bins', 'z_t'))*365

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
X_ctrl, Y_ctrl = np.meshgrid(ctrl.TLAT_bins[11:], -tdepth/1e3)
maxv = 5e11
plt.pcolormesh(X_ctrl, Y_ctrl, ctrl_zonal_levels_trend.T, cmap='RdBu_r', vmin=-maxv, vmax=maxv)
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.colorbar()
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_levels_trend_ctrl')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
X_rcp, Y_rcp = np.meshgrid(rcp.TLAT_bins[11:], -tdepth/1e3)
maxv = 5e11
plt.pcolormesh(X_rcp, Y_rcp, rcp_zonal_levels_trend.T, cmap='RdBu_r', vmin=-maxv, vmax=maxv)
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.colorbar()
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_levels_trend_rcp')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
X_rcp, Y_rcp = np.meshgrid(rcp.TLAT_bins[11:], -tdepth/1e3)
maxv = 10e11
plt.pcolormesh(X_rcp, Y_rcp, (rcp_zonal_levels_trend-ctrl_zonal_levels_trend).T, cmap='RdBu_r', vmin=-maxv, vmax=maxv)
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.colorbar()
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_levels_trend_rcp-ctrl')

In [ ]:
# hiatus
X_rcp, Y_rcp = np.meshgrid(rcp.TLAT_bins, -tdepth[:20]/1e3)
maxv=8e13
plt.pcolormesh(X_rcp, Y_rcp, (rcp.OHC_zonal_levels[28,:20,:]-rcp.OHC_zonal_levels[20,:20,:]), cmap='RdBu_r', vmin=-maxv, vmax=maxv)
plt.colorbar()
plt.tight_layout()